In [1]:
#%load_ext autoreload
#%autoreload 2
%matplotlib inline


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
#
# https://en.wikipedia.org/wiki/ISO_8601
import datetime
timestr = datetime.datetime.now().date().isoformat()
timestr = '2020-11-23'
tau = 9e-4

import sys
sys.path.append('..')
from HOTS.Event import conv2eve
from HOTS.Tools import SaveObject, LoadObject

ds = 10
ds = 20
NbClusteringData = 1600//ds
NbTrainingData = 1600//ds
NbTestingData = 1600//ds

records_path = '../Records'

def get_nmnist(timestr, NbTrainingData, NbTestingData, NbClusteringData, DataPath='../Data/testsetnmnist.p', verbose=False):
    fname_event_nmnist = f'{records_path}/EXP_03_NMNIST/{timestr}_{NbTrainingData}_{NbTestingData}_{NbClusteringData}_hots_event_nmnist.pkl'
    # print(help(LoadNMNIST))
    try:
        dataset = LoadObject(fname_event_nmnist)
        if verbose: print('loading the events from file', fname_event_nmnist)
    except:
        from HOTS.Event import LoadNMNIST
        dataset = LoadNMNIST(NbTrainingData, NbTestingData, NbClusteringData,
                             Path=DataPath, verbose=0)
        SaveObject(dataset, fname_event_nmnist)
        if verbose: print('saving the events to file', fname_event_nmnist)

    events_train, events_test, event_cluster, labels_train, labels_test = dataset
    return events_train, events_test, event_cluster, labels_train, labels_test

def get_events(timestr, NbTrainingData=NbTrainingData, NbTestingData=NbTestingData, NbClusteringData=NbClusteringData,
               tau=tau, N_layer=3, # -> tau=1ms, si on prend 10 ms on est à 1s pour la dernière couche et les vidéos font 0.3s en moyenne
               homeo=True, homrun = False, verbose=False):
    R = 2
    filthr = 2
    nbkNN = 3
    algo = 'lagorce'
    decay = 'exponential'
    krnlinit = 'rdn'
    #nb_cluster = [4, 8, 16]
    nb_cluster = [4*(2**i_layer) for i_layer in range(N_layer)]

    fname_ = f'{records_path}/EXP_03_NMNIST/{timestr}_hots_{tau*1000}_ms_{NbTrainingData}_{NbTestingData}_{NbClusteringData}'

    label = '_homeo' if homeo else ''

    fname_model = fname_ + '_model_' + algo + label + '.pkl'
    fname_event0_o = fname_ + '_event_out_' + algo + label + '.pkl'

    events_train, events_test, event_cluster, labels_train, labels_test = get_nmnist(timestr, NbTrainingData, NbTestingData, NbClusteringData)
    
    if not os.path.isfile(fname_event0_o):
        if verbose: print('creating the events in file', fname_event0_o)
        from HOTS.Event import Event


        from HOTS.Layer import ClusteringLayer
        opts_layer = dict(verbose=0, ThrFilter=filthr, LearningAlgo=algo, kernel=decay, homeo=homeo, init=krnlinit)
        layers = []
        for i_layer in range(N_layer):
            layers.append(ClusteringLayer(tau=(10**i_layer)*tau, R=(2**i_layer)*R, **opts_layer))

        from HOTS.Network import Network
        Net = Network(layers)#[L1, L2, L3])

        print(f'{os.getcwd()=}')
        print(f'{os.path.isfile(fname_model)=}')
        if not os.path.isfile(fname_model):
            if verbose: print('learning model', fname_model)
            ClusterLayer, event_output = Net.TrainCluster(
                    event=event_cluster, NbClusterList=nb_cluster, to_record=True, NbCycle=1
                )
            SaveObject(ClusterLayer, fname_model)
        else:
            if verbose: print('loading model from file', fname_model)

            ClusterLayer, Classif0 = LoadObject(fname_model)

        if verbose: print('run the events through the network')

        events_train_o = Net.RunNetwork(events_train, NbClusterList=ClusterLayer, homrun=homrun)
        events_test_o = Net.RunNetwork(events_test, NbClusterList=ClusterLayer, homrun=homrun)
        SaveObject([events_train, events_test], fname_event0_o)
    else:
        if verbose: print('loading the events from file', fname_event0_o)
        events_train_o, events_test_o = LoadObject(fname_event0_o)

    return events_train_o, events_test_o, labels_train, labels_test


# ### Building matrix for logistic regression

def gather_data(events_in, labels_in,
                tau_cla=.150, # characteristic time of a digit
                sample_events=50, sample_space = 1, do_cumulative=False,
                verbose=False, debug=False):

    n_events = events_in.time.shape[0]

    c_int = lambda n, d : ((n - 1) // d) + 1
    data = np.zeros((c_int(events_in.ImageSize[0], sample_space),
                     c_int(events_in.ImageSize[1], sample_space),
                     len(events_in.ListPolarities))) #tmp data

    X = np.zeros((c_int(n_events, sample_events), len(data.ravel())))
    y = np.zeros((c_int(n_events, sample_events), ))

    for i_event in range(1, n_events):

        data *= np.exp(-(events_in.time[i_event]-events_in.time[i_event-1])/tau_cla)

        x_pos = events_in.address[i_event, 0]//sample_space
        y_pos = events_in.address[i_event, 1]//sample_space
        p = events_in.polarity[i_event]
        if do_cumulative:
            data[x_pos, y_pos, p] = 1.
        else:
            data[x_pos, y_pos, p] += 1.

        if i_event % sample_events == sample_events//2 :
            if debug:
                print(f'DEBUG {i_event=} {i_event//sample_events=} ')
                print(f'DEBUG {y[i_event//sample_events]=}   ')
                print(f'DEBUG  {labels_in[i_event]=} ')
            X[i_event//sample_events, :] = data.ravel()
            y[i_event//sample_events] = labels_in[i_event]


    if verbose: print('Number of events: ' + str(X.shape[0])+' - Number of features: ' + str(X.shape[1]))

    return X, y


# ### Performing logistic regression
#
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html
#


# from sklearn.linear_model import LogisticRegression as LR
from sklearn.linear_model import LogisticRegressionCV as LR
# 
opts_LR = dict(max_iter=2000, # random_state=0,
               n_jobs=-1, class_weight='balanced')
#opts_LR['Cs'] = 5
opts_LR['Cs'] = 32
from sklearn.model_selection import train_test_split


# ### Performing logistic regression on raw input

# In[8]:


def tic():
    global ttic
    ttic = time.time()
def toc():
    print(f'Done in {time.time() - ttic:.3f} s')


In [3]:
verbose=False

### Performing logistic regression on raw input spikes


In [4]:
tic()
dataset = get_nmnist(timestr, NbTrainingData, NbTestingData, NbClusteringData, verbose=True)
events_train, events_test, events_cluster, labels_train, labels_test = dataset
toc()

loading the events from file ../Records/EXP_03_NMNIST/2020-11-23_80_80_80_hots_event_nmnist.pkl
Done in 0.029 s


In [5]:
tic()
X_train, y_train = gather_data(events_train, labels_train, verbose=verbose)
toc()

Done in 2.842 s


In [6]:
tic()
lr = LR(**opts_LR).fit(X_train, y_train)
print(f'Classification score for raw input is {lr.score(X_train, y_train):.3f} (train)')
toc()

Classification score for raw input is 0.988 (train)
Done in 992.828 s


In [7]:
tic()
X_test, y_test = gather_data(events_test, labels_test, verbose=verbose)
print(f'Classification score for raw input is {lr.score(X_test, y_test):.3f} (test)')
toc()

Classification score for raw input is 0.574 (test)
Done in 2.455 s


### Performing logistic regression with or without homeostasis


In [8]:
for homeo in [True, False, ]:
    print(40*'-')
    print(f'{homeo=}')
    print(40*'-')
    events_train_o, events_test_o, labels_train, labels_test = get_events(timestr, tau=tau, homeo=homeo, verbose=verbose)

    X_train, y_train = gather_data(events_train_o, labels_train, verbose=verbose)
    lr = LR(**opts_LR, verbose=verbose).fit(X_train, y_train)
    print(f'Classification score for {homeo=} is {lr.score(X_train, y_train):.3f} (train)')
    
    X_test, y_test = gather_data(events_test_o, labels_test, verbose=verbose)
    print(f'Classification score for {homeo=} is {lr.score(X_test, y_test):.3f} (test)')

----------------------------------------
homeo=True
----------------------------------------
Classification score for homeo=True is 0.988 (train)
Classification score for homeo=True is 0.574 (test)
----------------------------------------
homeo=False
----------------------------------------
Classification score for homeo=False is 0.988 (train)
Classification score for homeo=False is 0.574 (test)


### Performing logistic regression using a cumulative synaptic input

This makes it more similar to a histogram


In [9]:
for homeo in [True, False, ]:
    print(40*'-')
    print(f'{homeo=}')
    print(40*'-')
    events_train_o, events_test_o, labels_train, labels_test = get_events(timestr, tau=tau, homeo=homeo, verbose=verbose)

    X_train, y_train = gather_data(events_train_o, labels_train, do_cumulative=True, verbose=verbose)
    lr = LR(**opts_LR, verbose=verbose).fit(X_train, y_train)
    print(f'Classification score for {homeo=} is {lr.score(X_train, y_train):.3f} (train)')
    
    X_test, y_test = gather_data(events_test_o, labels_test, verbose=verbose)
    print(f'Classification score for {homeo=} is {lr.score(X_test, y_test):.3f} (test)')

----------------------------------------
homeo=True
----------------------------------------
Classification score for homeo=True is 0.998 (train)
Classification score for homeo=True is 0.269 (test)
----------------------------------------
homeo=False
----------------------------------------
Classification score for homeo=False is 0.998 (train)
Classification score for homeo=False is 0.269 (test)


### Performing logistic regression with different solvers

In [10]:
for homeo in [False, True]:
    print(40*'-')
    print(f'{homeo=}')
    print(40*'-')
    for solver, penalty in zip(['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], ['l2', 'l2', 'l1', 'l2', 'l1']):
        events_train_o, events_test_o, labels_train, labels_test = get_events(timestr, tau=tau, homeo=homeo, verbose=verbose)

        tic = time.time()
        X_train, y_train = gather_data(events_train_o, labels_train, verbose=verbose)
        lr = LR(solver=solver, penalty=penalty, verbose=verbose, **opts_LR).fit(X_train, y_train)
        print(f'Classification score for {homeo=} is {lr.score(X_train, y_train):.3f} (train)')
        print(f'Fit done in {time.time() -tic:.3f} seconds for {solver=}')
        X_test, y_test = gather_data(events_test_o, labels_test, verbose=verbose)
        print(f'Classification score for {homeo=} / {solver=} is {lr.score(X_test, y_test):.3f} (test)')

----------------------------------------
homeo=False
----------------------------------------
Classification score for homeo=False is 0.988 (train)
Fit done in 978.314 seconds for solver='newton-cg'
Classification score for homeo=False / solver='newton-cg' is 0.573 (test)
Classification score for homeo=False is 0.988 (train)
Fit done in 854.962 seconds for solver='lbfgs'
Classification score for homeo=False / solver='lbfgs' is 0.574 (test)


/usr/local/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Classification score for homeo=False is 0.999 (train)
Fit done in 154.603 seconds for solver='liblinear'
Classification score for homeo=False / solver='liblinear' is 0.443 (test)
Classification score for homeo=False is 0.987 (train)
Fit done in 3173.017 seconds for solver='sag'
Classification score for homeo=False / solver='sag' is 0.578 (test)
Classification score for homeo=False is 0.982 (train)
Fit done in 7848.863 seconds for solver='saga'
Classification score for homeo=False / solver='saga' is 0.534 (test)
----------------------------------------
homeo=True
----------------------------------------
Classification score for homeo=True is 0.988 (train)
Fit done in 924.326 seconds for solver='newton-cg'
Classification score for homeo=True / solver='newton-cg' is 0.573 (test)
Classification score for homeo=True is 0.988 (train)
Fit done in 755.166 seconds for solver='lbfgs'
Classification score for homeo=True / solver='lbfgs' is 0.574 (test)


/usr/local/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Classification score for homeo=True is 1.000 (train)
Fit done in 141.457 seconds for solver='liblinear'
Classification score for homeo=True / solver='liblinear' is 0.436 (test)
Classification score for homeo=True is 0.987 (train)
Fit done in 2924.991 seconds for solver='sag'
Classification score for homeo=True / solver='sag' is 0.578 (test)
Classification score for homeo=True is 0.982 (train)
Fit done in 8143.520 seconds for solver='saga'
Classification score for homeo=True / solver='saga' is 0.534 (test)


In [11]:
for homeo in [False, True]:
    print(40*'-')
    print(f'{homeo=}')
    print(40*'-')
    events_train_o, events_test_o, labels_train, labels_test = get_events(timestr, tau=tau, homeo=homeo, verbose=verbose)

    for tau_cla in .15 * np.logspace(-1, 1, 7, base=4):

        X_train, y_train = gather_data(events_train_o, labels_train, tau_cla=tau_cla, verbose=verbose)
        lr = LR(**opts_LR, verbose=verbose).fit(X_train, y_train)
        print(f'Classification score for {tau_cla=:.3f} is {lr.score(X_train, y_train):.3f} (train)')
        X_test, y_test = gather_data(events_test_o, labels_test, tau_cla=tau_cla, verbose=verbose)
        print(f'Classification score for {tau_cla=:.3f} is {lr.score(X_test, y_test):.3f} (test)')

----------------------------------------
homeo=False
----------------------------------------
Classification score for tau_cla=0.037 is 0.968 (train)
Classification score for tau_cla=0.037 is 0.574 (test)
Classification score for tau_cla=0.060 is 0.980 (train)
Classification score for tau_cla=0.060 is 0.589 (test)
Classification score for tau_cla=0.094 is 0.984 (train)
Classification score for tau_cla=0.094 is 0.589 (test)
Classification score for tau_cla=0.150 is 0.988 (train)
Classification score for tau_cla=0.150 is 0.574 (test)
Classification score for tau_cla=0.238 is 0.986 (train)
Classification score for tau_cla=0.238 is 0.542 (test)
Classification score for tau_cla=0.378 is 0.987 (train)
Classification score for tau_cla=0.378 is 0.479 (test)
Classification score for tau_cla=0.600 is 0.993 (train)
Classification score for tau_cla=0.600 is 0.402 (test)
----------------------------------------
homeo=True
----------------------------------------
Classification score for tau_cla=0.0

In [12]:
for homeo in [False, True]:
    print(40*'-')
    print(f'{homeo=}')
    print(40*'-')
    events_train_o, events_test_o, labels_train, labels_test = get_events(timestr, tau=tau, homeo=homeo, verbose=verbose)

    for sample_space in [4, 2, 1]:
        X_train, y_train = gather_data(events_train_o, labels_train, sample_space=sample_space, verbose=verbose)
        lr = LR(**opts_LR, verbose=verbose).fit(X_train, y_train)
        print(f'Classification score for {sample_space=} is {lr.score(X_train, y_train):.3f} (train)')
        X_test, y_test = gather_data(events_test_o, labels_test, sample_space=sample_space, verbose=verbose)
        print(f'Classification score for {sample_space=} is {lr.score(X_test, y_test):.3f} (test)')


----------------------------------------
homeo=False
----------------------------------------
Classification score for sample_space=4 is 0.973 (train)
Classification score for sample_space=4 is 0.517 (test)
Classification score for sample_space=2 is 0.985 (train)
Classification score for sample_space=2 is 0.540 (test)
Classification score for sample_space=1 is 0.988 (train)
Classification score for sample_space=1 is 0.574 (test)
----------------------------------------
homeo=True
----------------------------------------
Classification score for sample_space=4 is 0.973 (train)
Classification score for sample_space=4 is 0.517 (test)
Classification score for sample_space=2 is 0.985 (train)
Classification score for sample_space=2 is 0.540 (test)
Classification score for sample_space=1 is 0.988 (train)
Classification score for sample_space=1 is 0.574 (test)


In [13]:
tauz = np.array([1e-5, 1e-4, 2e-4 ,3e-4, 4e-4, 5e-4, 6e-4, 7e-4,
                 8e-4, 9e-4, 1e-3, 2e-3, 2.5e-3, 3e-3, 4e-3, 5e-3, 
                 1e-2, 2e-2])
for homeo in [False, True]:
    print(40*'-')
    print(f'{homeo=}')
    print(40*'-')
    for tau_ in tauz:
        events_train_o, events_test_o, labels_train, labels_test = get_events(timestr, tau=tau_, homeo=homeo, verbose=verbose)
        
        X_train, y_train = gather_data(events_train_o, labels_train, verbose=verbose)
        lr = LR(**opts_LR, verbose=verbose).fit(X_train, y_train)
        print(f'Classification score for {homeo=} & {tau_=:.6f} is {lr.score(X_train, y_train):.3f} (train)')
        X_test, y_test = gather_data(events_test_o, labels_test, verbose=verbose)
        print(f'Classification score for {homeo=} & {tau_=:.6f} is {lr.score(X_test, y_test):.3f} (test)')


----------------------------------------
homeo=False
----------------------------------------
Classification score for homeo=False & tau_=0.000010 is 0.988 (train)
Classification score for homeo=False & tau_=0.000010 is 0.574 (test)
Classification score for homeo=False & tau_=0.000100 is 0.988 (train)
Classification score for homeo=False & tau_=0.000100 is 0.574 (test)
Classification score for homeo=False & tau_=0.000200 is 0.988 (train)
Classification score for homeo=False & tau_=0.000200 is 0.574 (test)
Classification score for homeo=False & tau_=0.000300 is 0.988 (train)
Classification score for homeo=False & tau_=0.000300 is 0.574 (test)
Classification score for homeo=False & tau_=0.000400 is 0.988 (train)
Classification score for homeo=False & tau_=0.000400 is 0.574 (test)
Classification score for homeo=False & tau_=0.000500 is 0.988 (train)
Classification score for homeo=False & tau_=0.000500 is 0.574 (test)
Classification score for homeo=False & tau_=0.000600 is 0.988 (train)
Cl